# **Import Module**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf
import math
import random

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, GRU, Attention
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers

warnings.filterwarnings('ignore')

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# **Download Data**

In [2]:
!wget --no-check-certificate https://raw.githubusercontent.com/rioooranteai/data-analytics---modeling/main/Apple%20Stock/Dataset/apple_stock_data.csv -O /content/apple_stock_data.csv

--2025-03-18 15:02:34--  https://raw.githubusercontent.com/rioooranteai/data-analytics---modeling/main/Apple%20Stock/Dataset/apple_stock_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31516 (31K) [text/plain]
Saving to: ‘/content/apple_stock_data.csv’

/content/apple_stoc 100%[===================>]  30.78K  --.-KB/s    in 0.01s   

2025-03-18 15:02:35 (2.83 MB/s) - ‘/content/apple_stock_data.csv’ saved [31516/31516]



# **Read Data**

In [3]:
df = pd.read_csv('/content/apple_stock_data.csv')
df.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       252 non-null    object 
 1   Adj Close  252 non-null    float64
 2   Close      252 non-null    float64
 3   High       252 non-null    float64
 4   Low        252 non-null    float64
 5   Open       252 non-null    float64
 6   Volume     252 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB


In [5]:
df.describe()

,Adj Close,Close,High,Low,Open,Volume
count,252.000000,252.000000,252.000000,252.000000,252.000000,2.520000e+02
mean,199.088202,199.454286,201.093056,197.608373,199.316032,5.832214e+07
std,21.511933,21.321567,21.571427,20.963034,21.365476,3.025771e+07
min,164.585999,165.000000,166.399994,164.080002,165.350006,2.404830e+07
25%,182.254063,182.852501,184.617504,181.487499,182.777504,4.278855e+07
50%,192.370026,193.084999,194.399994,191.724998,192.989998,5.174865e+07
75%,221.697502,221.697502,224.149994,219.717499,221.635002,6.497460e+07
max,236.479996,236.479996,237.490005,234.449997,236.479996,3.186799e+08


# **Module 1**
## **Kishan Manani - Feature Engineering for Time Series Forecasting | PyData London 2022**
- [Video Link](https://youtu.be/9QtL7m3YS9I?si=D-aF93y8QnVh36TF)
- [Github](https://github.com/KishManani/PyDataLondon2022)

In [15]:
data_time_series = df[['Date','Close']].copy(deep=True)
data_time_series.set_index('Date', inplace=True)
data_time_series.index = pd.to_datetime(data_time_series.index)
data_time_series.head()

,Close
Date,
2023-11-02 00:00:00+00:00,177.570007
2023-11-03 00:00:00+00:00,176.649994
2023-11-06 00:00:00+00:00,179.229996
2023-11-07 00:00:00+00:00,181.820007
2023-11-08 00:00:00+00:00,182.889999


## **Time Series to a Table of Features and Target**

Teknik Feature Engineering untuk pemodelan *time series forecasting*:

1. **Static Feature**  
   - **Makna**: Kolom ini merepresentasikan atribut yang tidak berubah seiring waktu. Misalnya negara (UK, USA, dsb.) atau kategori lain yang melekat pada data secara permanen.  
   - **Konteks Forecasting**: Meskipun tidak berubah dari waktu ke waktu, fitur statis dapat memengaruhi pola perilaku target. Sebagai contoh, “Negara” bisa memengaruhi kebiasaan konsumen, musim (iklim), atau faktor-faktor lain yang spesifik pada lokasi tersebut.

2. **Feature dengan Nilai Tidak Diketahui di Masa Depan**  
   - **Makna**: Kolom ini adalah contoh *exogenous feature* (variabel eksternal) yang memengaruhi target (Y). Contoh: curah hujan memengaruhi penjualan payung atau produk tertentu.  
   - **Konteks Forecasting**: Nilai di masa depan belum tentu tersedia atau pasti, kecuali kita punya prediksi cuaca. Jika tidak ada model prediksi cuaca, maka kita tidak dapat secara langsung menggunakan nilainya untuk meramalkan masa depan (karena kita *tidak tahu* persis nilai Rainfall di masa depan).  
   - **Catatan**: Kadang kita bisa menggabungkan *forecast* cuaca (jika tersedia) sehingga “Rainfall” bisa menjadi “dikenal” di masa depan. Namun, jika tidak, kita perlu strategi khusus (misalnya *scenario-based forecasting* atau mengabaikan kolom ini untuk prediksi masa depan).

3. **Feature dengan Nilai Diketahui di Masa Depan**  
   - **Makna**: Kolom ini adalah contoh variabel eksternal yang dapat kita kontrol atau rencanakan. Misalnya, anggaran iklan (ad spend) yang sudah ditetapkan oleh tim marketing untuk periode mendatang.  
   - **Konteks Forecasting**: Karena kita “tahu” berapa besar biaya iklan di masa depan (misalnya sudah dijadwalkan), maka kolom ini bisa diperlakukan sebagai *known future feature*. Model dapat memanfaatkannya untuk meningkatkan akurasi prediksi, karena variabel ini biasanya berpengaruh pada penjualan, *demand*, atau target lain.

4. **Lag Features**  
   - **Makna**: Kolom-kolom ini adalah nilai target di masa lalu, yang ditambahkan sebagai fitur. Contohnya:  
     - Yᵗ⁻¹: Nilai target (misalnya penjualan) satu periode sebelumnya.  
     - Yᵗ⁻²: Nilai target dua periode sebelumnya.  
     - Dan seterusnya.  
   - **Konteks Forecasting**: Lag features ini membantu model “mengingat” pola masa lalu, seperti tren, musiman, atau *autocorrelation* (ketergantungan nilai target pada nilai sebelumnya). Dengan memasukkan lag-lag ini, model dapat belajar hubungan antar waktu dan menghasilkan prediksi yang lebih akurat.

---
### Bagaimana Masing-Masing Fitur Digunakan dalam Model?
1. **Static Features (Contoh: Country)**: Di-*encode* (misalnya *one-hot encoding*) atau disimpan sebagai *embedding* agar model memahami perbedaan karakteristik antar kategori.  
2. **Unknown Future Features (Contoh: Rainfall)**: Biasanya:
   - Menggunakan prediksi eksternal (jika tersedia), atau  
   - Mengabaikannya untuk prediksi masa depan, atau  
   - Melakukan analisis skenario (memasukkan beberapa skenario nilai Rainfall).  
3. **Known Future Features (Contoh: Ad Spend)**: Dapat langsung dimasukkan sebagai input model untuk periode prediksi karena nilainya sudah kita ketahui.  
4. **Lag Features (Yᵗ⁻³, Yᵗ⁻², Yᵗ⁻¹, Yᵗ)**: Membangun *autoregressive* pattern agar model belajar perilaku target berdasarkan nilai historisnya.

Dengan cara inilah tabel di atas merepresentasikan berbagai jenis fitur yang dipertimbangkan dalam *time series forecasting*. Semoga membantu!

In [31]:
def lag_features(df, n_lookback):
    X = []
    y = []

    for i in range(len(df) - n_lookback):
        X.append(df[i:i+n_lookback].values.flatten())
        y.append(df[i+n_lookback])

    df_x = pd.DataFrame(X, columns=[f'Yt-{i}' for i in range(n_lookback, 0, -1)])
    df_y = pd.DataFrame(y, columns=['Y'])

    df_x = df_x.reset_index(drop=True)
    df_y = df_y.reset_index(drop=True)

    df = pd.concat([df_x, df_y], axis=1)

    return df

In [33]:
lag_features(data_time_series['Close'], 5)

,Yt-5,Yt-4,Yt-3,Yt-2,Yt-1,Y
0,177.570007,176.649994,179.229996,181.820007,182.889999,182.410004
1,176.649994,179.229996,181.820007,182.889999,182.410004,186.399994
2,179.229996,181.820007,182.889999,182.410004,186.399994,184.800003
3,181.820007,182.889999,182.410004,186.399994,184.800003,187.440002
4,182.889999,182.410004,186.399994,184.800003,187.440002,188.009995
...,...,...,...,...,...,...
242,236.479996,235.860001,230.759995,230.570007,231.410004,233.399994
243,235.860001,230.759995,230.570007,231.410004,233.399994,233.669998
244,230.759995,230.570007,231.410004,233.399994,233.669998,230.100006
245,230.570007,231.410004,233.399994,233.669998,230.100006,225.910004


# **Multi-step Forecasting**